In [ ]:
# Football Player and Ball Tracking 

import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
np.float = float  # monkey patch for compatibility

# --------------------------
# STEP 1: Configuration
# --------------------------
input_path = "input/15sec_input_720p.mp4"
output_path = "outputs/tracked_output.mp4"
model_path = "yolov.pt"  # YOLOv fallback

# --------------------------
# STEP 2: Load YOLOv model
# --------------------------
model = YOLO(model_path)
print("✅ YOLOv model loaded successfully")

# --------------------------
# STEP 3: Load DeepSORT with tuned parameters
# --------------------------
deep_sort_tracker = DeepSort(max_age=30, n_init=3, max_cosine_distance=0.3, nn_budget=100)
print("✅ DeepSORT tracker loaded successfully")

# --------------------------
# STEP 4: Video I/O setup
# --------------------------
cap = cv2.VideoCapture(input_path)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# --------------------------
# STEP 5: Define helper functions
# --------------------------
def get_color(track_id):
    np.random.seed(int(track_id))
    return tuple(np.random.randint(0, 255, 3).tolist())

track_id_map = {}
next_pid = 1

# --------------------------
# STEP 6: Process Video
# --------------------------
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    results = model(frame, verbose=False)[0]

    detections = []
    for r in results.boxes:
        cls_id = int(r.cls.item())
        conf = r.conf.item()
        if conf < 0.3:
            continue
        xyxy = r.xyxy.cpu().numpy().astype(int)[0]
        bbox_ltwh = [xyxy[0], xyxy[1], xyxy[2]-xyxy[0], xyxy[3]-xyxy[1]]
        detections.append((bbox_ltwh, conf, cls_id))

    tracks = deep_sort_tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        tid = track.track_id
        if tid not in track_id_map:
            track_id_map[tid] = f"P{next_pid}"
            next_pid += 1

        label = track_id_map[tid]
        l, t, w, h = track.to_ltrb()
        l, t, r, b = int(l), int(t), int(l + w), int(t + h)

        color = get_color(tid)
        cv2.rectangle(frame, (l, t), (r, b), color, 2)
        cv2.putText(frame, label, (l, t - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(frame)
    if frame_count % 30 == 0:
        print(f"Processed {frame_count} frames")

cap.release()
out.release()
print("✅ Tracking complete! Output saved at", output_path)


✅ YOLOv model loaded successfully


Ultralytics YOLOv8.0.20  Python-3.10.18 torch-2.1.0+cpu CPU


✅ DeepSORT tracker loaded successfully


YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


Processed 30 frames
Processed 60 frames
Processed 90 frames
Processed 120 frames
Processed 150 frames
Processed 180 frames
Processed 210 frames
Processed 240 frames
Processed 270 frames
Processed 300 frames
Processed 330 frames
Processed 360 frames
✅ Tracking complete! Output saved at outputs/tracked_output.mp4
